In [1]:
import geopandas
import scipy.io
import numpy as np
import pandas as pd
import xarray as xr
from EchoPro import EchoPro
epro_2019 = EchoPro(init_file_path='./config_files/initialization_config.yml',
                    survey_year_file_path='./config_files/survey_year_2019_config.yml',
                    source=3,
                    bio_data_type=1,
                    age_data_status=1, 
                    exclude_age1=True)

A check of the initialization file needs to be done!
A check of the survey year file needs to be done!
Loading biological data ...
getting strata data
Do we need to set stratum_id or just use strata_df? Look into this!


/Users/brandonreyes/UW_work/EchoPro_work/EchoPro/EchoPro/echo_pro.py:275: UserWarning: We are currently using nwgt_total from Matlab for CV, change this!
  warnings.warn("We are currently using nwgt_total from Matlab for CV, change this!")


In [2]:
# get geopandas representation of final_biomass_table
# allows us to plot transect data
df = epro_2019.final_biomass_table
gdf = geopandas.GeoDataFrame(df, 
                             geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

# reseting the index so that we can select data based on column name
gdf = gdf.reset_index()

## Generate keys

In [3]:
# import class to use it's functions 
from EchoPro.load_stratification_data import LoadStrataData

strata_class = LoadStrataData(epro_2019)

# get df relating the haul to the stratum
strata_haul_df = epro_2019.strata_df.reset_index()[['Haul', 'strata']].set_index('Haul')

# get the bins for the lengths
bins_len = epro_2019.params['bio_hake_len_bin']

# get the bins for the ages
bins_age = epro_2019.params['bio_hake_age_bin']

# get all specimen data that is necessary for key generation
spec_w_strata = epro_2019.specimen_df.drop('Specimen_Number', axis=1).copy().reset_index()

# add strata column 
spec_w_strata['Strata'] = spec_w_strata.apply(lambda x: strata_haul_df.loc[x[0]], 
                                              axis=1).values

spec_w_strata.set_index('Strata', inplace=True)

# spec_w_strata = spec_w_strata[(spec_w_strata['Sex'] != 3)].copy() # TODO: this should be for all sexes

age_len_key_da, age_len_key_wgt_da, age_len_key_norm_da = strata_class.get_age_key_das(spec_w_strata, 
                                                                                       bins_len, bins_age)

# TODO: it would probably be better to do an average of station 1 and 2 here... (Chu doesn't do this)
age_len_key_wgt_norm_da = age_len_key_wgt_da/age_len_key_wgt_da.sum(dim=['len_bins', 'age_bins'])

# each stratum's multiplier once normalized weight has been calculated
age2_wgt_proportion_da = 1.0 - age_len_key_wgt_norm_da.isel(age_bins=0).sum(dim='len_bins')/age_len_key_wgt_norm_da.sum(dim=['len_bins', 'age_bins'])

In [4]:
# get all specimen data that is necessary for key generation
spec_w_strata = epro_2019.specimen_df.drop('Specimen_Number', axis=1).copy().reset_index()

# add strata column 
spec_w_strata['Strata'] = spec_w_strata.apply(lambda x: strata_haul_df.loc[x[0]], 
                                              axis=1).values

spec_w_strata.set_index('Strata', inplace=True)

reg_w0, reg_p = strata_class.get_length_val_reg_vals(len_name='Length', val_name="Weight", df=spec_w_strata)

len_weight_ALL, len_nALL, norm_len_key_ALL = strata_class.generate_length_val_key(bins_len, reg_w0, reg_p, 
                                                                               len_name='Length',
                                                                                 val_name='Weight', df=spec_w_strata)

# spec_w_strata = spec_w_strata[(spec_w_strata['Sex'] != 3)] # TODO: this should be for all sexes
len_wgt_key_spec_da, len_key_spec_da, len_key_norm_spec_da = strata_class.get_weight_key_das(spec_w_strata, 
                                                                                bins_len, reg_w0, reg_p, 
                                                                                len_name='Length',
                                                                                val_name='Weight')

In [6]:
length_explode_df = epro_2019.length_df[['Sex', 'Length']].copy()
# add strata column 
length_explode_df['Strata'] = length_explode_df.reset_index().apply(lambda x: strata_haul_df.loc[x[0]], 
                                                                    axis=1).values

length_explode_df.reset_index(inplace=True)

length_explode_df.set_index('Strata', inplace=True)

length_explode_df = length_explode_df.explode(['Sex', 'Length'])

length_explode_df = length_explode_df.astype({'Haul': int, 
                                              'Sex': int, 
                                              'Length': np.float64})

In [7]:

# length_explode_df = length_explode_df[(length_explode_df['Sex'] != 3)] # TODO: this should be for all sexes

unique_strata = length_explode_df.index.unique().values

len_key_norm_length = np.empty((unique_strata.shape[0], bins_len.shape[0]), dtype=np.float64)
len_key_norm_length[:, :] = 0.0

stratum_ind = 0
for stratum in unique_strata:
    input_data = length_explode_df.loc[stratum]['Length'].values
    len_ind = strata_class.get_bin_ind(input_data, bins_len)

    len_key_n = np.array([i.shape[0] for i in len_ind])
    len_key_norm_length[stratum_ind, :] = len_key_n/np.sum(len_key_n)
    
    stratum_ind += 1
    
    
len_key_norm_length_da = xr.DataArray(data=len_key_norm_length, 
                                      coords={'strata': unique_strata, 'len_bins': bins_len})

In [8]:
len_key_norm_length_da.sel(strata=1)

<xarray.DataArray (len_bins: 40)>
array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00125471, 0.18067754,
       0.50690088, 0.24090339, 0.03136763, 0.0238394 , 0.01003764,
       0.00376412, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00125471,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])
Coordinates:
    strata    int64 1
  * len_bins  (len_bins) int64 2 4 6 8 10 12 14 16 ... 66 68 70 72 74 76 78 80

In [9]:
len_key_norm_spec_da.sel(strata=1)

<xarray.DataArray (len_bins: 40)>
array([0.        , 0.        , 0.        , 0.        , 0.00502513,
       0.        , 0.        , 0.        , 0.        , 0.18592965,
       0.50251256, 0.24623116, 0.04020101, 0.00502513, 0.01005025,
       0.        , 0.00502513, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])
Coordinates:
    strata    int64 1
  * len_bins  (len_bins) int64 2 4 6 8 10 12 14 16 ... 66 68 70 72 74 76 78 80

In [10]:
len_key_norm_ave = (len_key_norm_length_da + len_key_norm_spec_da)/2

In [11]:
len_key_norm_ave.sel(strata=1)

<xarray.DataArray (len_bins: 40)>
array([0.        , 0.        , 0.        , 0.        , 0.00251256,
       0.        , 0.        , 0.        , 0.00062735, 0.18330359,
       0.50470672, 0.24356727, 0.03578432, 0.01443226, 0.01004395,
       0.00188206, 0.00251256, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00062735,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])
Coordinates:
    strata    int64 1
  * len_bins  (len_bins) int64 2 4 6 8 10 12 14 16 ... 66 68 70 72 74 76 78 80

## Calculate the biomass density

In [12]:
# get the nasc dataframe 
nasc_df = epro_2019.load_nasc_data()

# calculates the interval for the area calculation
interval = (nasc_df['VL start'].iloc[1:].values - nasc_df['VL start'].iloc[:-1].values)
last_interval = nasc_df['VL end'].iloc[-1] - nasc_df['VL start'].iloc[-1]

interval = np.concatenate([interval, np.array([last_interval])])

median_interval = np.median(interval)

# remove outliers at the end of the transect
ind_outliers = np.argwhere(np.abs(interval - median_interval) > 0.05).flatten()
interval[ind_outliers] = nasc_df['VL end'].values[ind_outliers] - nasc_df['VL start'].values[ind_outliers]

In [13]:
bio_dense_df = nasc_df[['Stratum', 'NASC', 'Haul']].copy()
bio_dense_df['interval'] = interval
bio_dense_df['n_A'] = nasc_df.apply(lambda x: x.NASC/epro_2019.strata_ds.sig_b.sel(strata=x.Stratum).values, axis=1)
# bio_dense_df['A'] = bio_dense_df['interval']*nasc_df['Spacing']
# bio_dense_df['N_A'] = bio_dense_df['n_A']*bio_dense_df['A']

In [14]:
bio_density = bio_dense_df.apply(lambda x: x.n_A*np.dot(len_key_norm_ave.sel(strata=x.Stratum), 
                                          len_wgt_key_spec_da.sel(strata=x.Stratum)), axis=1)

bio_density_2_prop = bio_dense_df.apply(lambda x: x.n_A*np.dot(len_key_norm_ave.sel(strata=x.Stratum), 
                                        len_wgt_key_spec_da.sel(strata=x.Stratum))*age2_wgt_proportion_da.sel(strata=x.Stratum).values,
                                        axis=1)

In [15]:
bio_density.reset_index().loc[750: 760]

,Transect,0
750,9,0.000000
751,9,0.000000
752,9,0.000000
753,9,0.000000
754,9,423.791125
755,9,2326.019570
756,9,4352.792496
757,9,2758.018092
758,9,4040.597961
759,9,3134.167289


In [16]:
bio_density_2_prop.reset_index().loc[750: 760]

,Transect,0
750,9,0.000000
751,9,0.000000
752,9,0.000000
753,9,0.000000
754,9,241.266499
755,9,1324.215082
756,9,2478.067487
757,9,1570.154095
758,9,2300.333509
759,9,1784.297797


In [17]:
epro_2019.final_biomass_table["nwgt_total"].reset_index().loc[750: 760]

,Transect,nwgt_total
750,9,0.000000
751,9,0.000000
752,9,0.000000
753,9,0.000000
754,9,232.112442
755,9,1273.975595
756,9,2384.025103
757,9,1510.564750
758,9,2213.043001
759,9,1716.601410


In [18]:
arr1 = epro_2019.final_biomass_table["nwgt_total"].values
arr2 = bio_density_2_prop.values

In [19]:
out = (np.abs(arr1 - arr2)/((arr1 + arr2)/2.0))*100

/var/folders/68/bd5dqh4j3zgbwmhw2_9v4g180000gn/T/ipykernel_53672/3926264985.py:1: RuntimeWarning: invalid value encountered in true_divide
  out = (np.abs(arr1 - arr2)/((arr1 + arr2)/2.0))*100


In [20]:
no_nan_ind = np.logical_not(np.isnan(out)).flatten()

np.mean(out[no_nan_ind])

5.026445532085591

In [21]:
# %%time
# lat_INPFC = [np.NINF, 36, 40.5, 43.000, 45.7667, 48.5, 55.0000]  # INPFC
# epro_2019.run_cv_analysis(lat_INPFC, kriged_data=False)

In [22]:
# CV_JH_mean = 0.13377365505687697
# CPU times: user 2min 31s, sys: 312 ms, total: 2min 31s
# Wall time: 2min 31s